In [37]:
import numpy as np 
import cv2

In [38]:
# import intrinsic matrix and distortion coefficients
intrinsic_matrix = np.load('../intrinsic_matrices/1.npy')
distortion_coefficients = np.load('../distortions/1.npy')

In [39]:
'''
# cut out single frame
import moviepy.editor as mpy
maskclip = mpy.VideoFileClip("../videos/out1.mp4")
maskclip.save_frame("../extrinsic_calibration_images/out1.png", t='00:00:56')
'''

'\n# cut out single frame\nimport moviepy.editor as mpy\nmaskclip = mpy.VideoFileClip("../videos/out1.mp4")\nmaskclip.save_frame("../extrinsic_calibration_images/out1.png", t=\'00:00:56\')\n'

In [40]:
# generate world points
world_points = np.array([[0, 0, 0], [6, 0, 0], [0, 9, 0], [6, 9, 0]], dtype=np.float32) * 1000. # mm
image_points = np.array([[1482, 966], [2410, 1006], [2002, 1625], [3221, 1590]], dtype=np.float32) # px

In [47]:
# calibrate extrinsic matrix
success, rvecs, tvecs = cv2.solvePnP(world_points, image_points, intrinsic_matrix, distortion_coefficients)
rotation_matrix, _ = cv2.Rodrigues(rvecs)

# convert from 3x3 to 4x4
rotation_matrix = np.pad(rotation_matrix, (0, 1), mode='constant')
rotation_matrix[3, 3] = 1

tvecs = tvecs.flatten() / 1000.

t_matrix = np.array([[1, 0, 0, tvecs[0]],
                     [0, 1, 0, tvecs[1]],
                     [0, 0, 1, tvecs[2]],
                     [0, 0, 0, 1]])

world_f_camera = np.dot(t_matrix, rotation_matrix)
extrinsic_matrix = np.linalg.inv(world_f_camera)
print(f'Extrinsic Matrix:\n{extrinsic_matrix}')

Extrinsic Matrix:
[[ 0.9295526  -0.05298937  0.36486174 -6.76736156]
 [ 0.34081979  0.5009434  -0.7955486  24.58670986]
 [-0.14061946  0.86385638  0.48371306 -7.48816262]
 [ 0.          0.          0.          1.        ]]
